In [2]:
# importing some libraries we need in this part
import scipy.io
import numpy as np
import os
from collections import defaultdict, Counter
import pandas as pd
from matplotlib import pyplot as plt
import pickle
from cmath import pi
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
from scipy.signal import stft
from scipy import signal
import numpy as np


In [3]:
# Load your dataset
with open('Data/trainset.pickle', 'rb') as handle:
    trainset = pickle.load(handle)

# Inspect the shapes of each CSI entry
CSI_signals = trainset['CSI']
shapes = [CSI.shape for CSI in CSI_signals]
print(shapes)  # This will show the shapes of each CSI entry
target_length = 6900 


[(6600, 245, 2, 2), (6000, 245, 2, 2), (6500, 245, 2, 2), (6400, 245, 2, 2), (6700, 245, 2, 2), (6900, 245, 2, 2), (3400, 245, 2, 2), (4300, 245, 2, 2)]


In [4]:
import numpy as np
import pickle

# Load the dataset
with open('Data/trainset.pickle', 'rb') as handle:
    trainset = pickle.load(handle)

# Extract CSI signals and groundtruth
CSI_signals = trainset['CSI']
groundtruth = trainset['groundtruth']

# Function to process CSI signals
def process_CSI_signals(CSI_signals, target_length):
    processed_signals = []
    for signal in CSI_signals:
        signal = np.array(signal)
        if signal.shape[0] > target_length:
            # Truncate
            signal_processed = signal[:target_length]
        else:
            # Pad
            padding = target_length - signal.shape[0]
            signal_processed = np.pad(signal, ((0, padding), (0, 0), (0, 0), (0, 0)), mode='constant', constant_values=0)
        processed_signals.append(signal_processed)
    return np.array(processed_signals)

def process_groundtruth(groundtruth, target_length):
    processed_groundtruth = []
    for gt in groundtruth:
        gt = np.array(gt)  # Ensure groundtruth entry is a numpy array
        print(f'Original Groundtruth Length: {gt.shape[0]}')  # Debugging output
        if gt.shape[0] > target_length:
            # Truncate
            gt_processed = gt[:target_length]
        elif gt.shape[0] < target_length and gt.shape[0] > 2:
            # Pad only if greater than 2
            padding = target_length - gt.shape[0]
            gt_processed = np.pad(gt, (0, padding), mode='constant', constant_values=0)
        else:
            print("Skipping short groundtruth entry.")
            continue  # Skip entries that are too short
        print(f'Processed Groundtruth Length: {gt_processed.shape[0]}')  # Debugging output
        processed_groundtruth.append(gt_processed)
    return np.array(processed_groundtruth)

# Process the CSI signals and groundtruth
CSI_signals_processed = process_CSI_signals(CSI_signals, target_length)
groundtruth_processed = process_groundtruth(groundtruth, target_length)

# Check the shapes
print("CSI Signals Shape: ", CSI_signals_processed.shape)
# print("Groundtruth Shape: ", groundtruth_processed.shape)




# # Process the CSI signals and groundtruth
# CSI_signals_processed = process_CSI_signals(CSI_signals, target_length)
# groundtruth_processed = process_groundtruth(groundtruth, target_length)


# # Check the shapes
# print("Groundtruth Shape: ", CSI_signals_processed.shape)
# print("Groundtruth Shape: ", groundtruth_processed.shape)

# Now you can proceed with your deep learning model setup


Original Groundtruth Length: 66
Processed Groundtruth Length: 6900
Original Groundtruth Length: 60
Processed Groundtruth Length: 6900
Original Groundtruth Length: 65
Processed Groundtruth Length: 6900
Original Groundtruth Length: 64
Processed Groundtruth Length: 6900
Original Groundtruth Length: 67
Processed Groundtruth Length: 6900
Original Groundtruth Length: 69
Processed Groundtruth Length: 6900
Original Groundtruth Length: 2
Skipping short groundtruth entry.
Original Groundtruth Length: 2
Skipping short groundtruth entry.
CSI Signals Shape:  (8, 6900, 245, 2, 2)


In [5]:
import pickle

# Load the dataset
with open('Data/trainset.pickle', 'rb') as handle:
    trainset = pickle.load(handle)

# Print the keys
print("Keys in the dataset:", trainset.keys())

# Check the shape and structure of CSI and groundtruth
CSI_sample = trainset['CSI']
groundtruth_sample = trainset['groundtruth']

print("Sample CSI shape:", len(CSI_sample), "Entries")
print("Sample Groundtruth shape:", len(groundtruth_sample), "Entries")


Keys in the dataset: dict_keys(['ex_name', 'CSI', 'CSI_timestamp', 'CSI_info', 'groundtruth'])
Sample CSI shape: 8 Entries
Sample Groundtruth shape: 8 Entries


In [6]:
# Inspect the shape and content of each CSI entry and corresponding ground truth
for i in range(len(CSI_sample)):
    print(f"CSI Entry {i} shape: {CSI_sample[i].shape}")  # Print the shape of each CSI entry
    print(f"Groundtruth Entry {i} length: {len(groundtruth_sample[i])}")  # Print the length of each ground truth entry



CSI Entry 0 shape: (6600, 245, 2, 2)
Groundtruth Entry 0 length: 66
CSI Entry 1 shape: (6000, 245, 2, 2)
Groundtruth Entry 1 length: 60
CSI Entry 2 shape: (6500, 245, 2, 2)
Groundtruth Entry 2 length: 65
CSI Entry 3 shape: (6400, 245, 2, 2)
Groundtruth Entry 3 length: 64
CSI Entry 4 shape: (6700, 245, 2, 2)
Groundtruth Entry 4 length: 67
CSI Entry 5 shape: (6900, 245, 2, 2)
Groundtruth Entry 5 length: 69
CSI Entry 6 shape: (3400, 245, 2, 2)
Groundtruth Entry 6 length: 2
CSI Entry 7 shape: (4300, 245, 2, 2)
Groundtruth Entry 7 length: 2


In [7]:
import pickle

# Load the dataset
with open('Data/trainset.pickle', 'rb') as handle:
    trainset = pickle.load(handle)

# Check the keys in the dataset
print("Keys in the dataset:", trainset.keys())

# Extract the shapes of the CSI signals and groundtruth
CSI_signals = trainset['CSI']
groundtruth = trainset['groundtruth']

# Check the number of entries and their shapes
print(f"Number of CSI Entries: {len(CSI_signals)}")
for idx, signal in enumerate(CSI_signals):
    print(f"CSI Entry {idx} shape: {signal.shape}")

print(f"Number of Groundtruth Entries: {len(groundtruth)}")
for idx, gt in enumerate(groundtruth):
    print(f"Groundtruth Entry {idx} length: {len(gt)}")


Keys in the dataset: dict_keys(['ex_name', 'CSI', 'CSI_timestamp', 'CSI_info', 'groundtruth'])
Number of CSI Entries: 8
CSI Entry 0 shape: (6600, 245, 2, 2)
CSI Entry 1 shape: (6000, 245, 2, 2)
CSI Entry 2 shape: (6500, 245, 2, 2)
CSI Entry 3 shape: (6400, 245, 2, 2)
CSI Entry 4 shape: (6700, 245, 2, 2)
CSI Entry 5 shape: (6900, 245, 2, 2)
CSI Entry 6 shape: (3400, 245, 2, 2)
CSI Entry 7 shape: (4300, 245, 2, 2)
Number of Groundtruth Entries: 8
Groundtruth Entry 0 length: 66
Groundtruth Entry 1 length: 60
Groundtruth Entry 2 length: 65
Groundtruth Entry 3 length: 64
Groundtruth Entry 4 length: 67
Groundtruth Entry 5 length: 69
Groundtruth Entry 6 length: 2
Groundtruth Entry 7 length: 2


In [8]:
import torch

import torch

# Function to process groundtruth using PyTorch
def process_groundtruth_pytorch(groundtruth, target_length):
    processed_groundtruth = []
    
    for gt in groundtruth:
        gt_tensor = torch.tensor(gt, dtype=torch.float32)  # Convert to tensor
        print(f'Original Groundtruth Length: {gt_tensor.shape[0]}')  # Debugging output

        # Check the length of groundtruth entry
        if gt_tensor.shape[0] == 2:  # Skip entries with length 2
            print('Skipping groundtruth entry due to insufficient length.')
            continue
        
        if gt_tensor.shape[0] < target_length:
            # Pad with zeros if the entry is shorter than the target length
            padding = target_length - gt_tensor.shape[0]
            gt_processed = torch.nn.functional.pad(gt_tensor, (0, padding), value=0)  # Pad with zeros
            print(f'Padding groundtruth entry to length: {target_length}')
        else:
            # If the entry is longer than the target length, truncate it
            gt_processed = gt_tensor[:target_length]

        print(f'Processed Groundtruth Length: {gt_processed.shape[0]}')  # Debugging output
        processed_groundtruth.append(gt_processed)

    # Stack the tensors only if we have processed groundtruth entries
    if processed_groundtruth:  # Check if the list is not empty
        return torch.stack(processed_groundtruth)  # Stack the list into a tensor
    else:
        raise ValueError("No valid groundtruth entries were processed.")

# Process the groundtruth
groundtruth_processed = process_groundtruth_pytorch(trainset['groundtruth'], 69)

# Check the final shape
print("Processed Groundtruth Shape: ", groundtruth_processed.shape)


Original Groundtruth Length: 66
Padding groundtruth entry to length: 69
Processed Groundtruth Length: 69
Original Groundtruth Length: 60
Padding groundtruth entry to length: 69
Processed Groundtruth Length: 69
Original Groundtruth Length: 65
Padding groundtruth entry to length: 69
Processed Groundtruth Length: 69
Original Groundtruth Length: 64
Padding groundtruth entry to length: 69
Processed Groundtruth Length: 69
Original Groundtruth Length: 67
Padding groundtruth entry to length: 69
Processed Groundtruth Length: 69
Original Groundtruth Length: 69
Processed Groundtruth Length: 69
Original Groundtruth Length: 2
Skipping groundtruth entry due to insufficient length.
Original Groundtruth Length: 2
Skipping groundtruth entry due to insufficient length.
Processed Groundtruth Shape:  torch.Size([6, 69])


/tmp/ipykernel_35856/4264095175.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  gt_tensor = torch.tensor(gt, dtype=torch.float32)  # Convert to tensor


In [9]:
import torch

# Function to process CSI signals using PyTorch in batches
def process_CSI_signals_pytorch(CSI_signals, target_length):
    processed_signals = []
    
    for signal in CSI_signals:
        # Convert to tensor with float32 (or float16 if needed)
        signal_tensor = torch.tensor(signal, dtype=torch.float32)  # Keep complex parts separate if needed
        print(f'Original CSI Signal Shape: {signal_tensor.shape}')  # Debugging output
        
        if signal_tensor.shape[0] > target_length:
            # Truncate
            signal_processed = signal_tensor[:target_length]
        else:
            # Pad
            padding = target_length - signal_tensor.shape[0]
            # Pad with zeros only on the first dimension
            signal_processed = torch.nn.functional.pad(signal_tensor, (0, 0, 0, padding), value=0)  # Pad with zeros
            print(f'Padding CSI signal to length: {target_length}')
        
        processed_signals.append(signal_processed)
        print(f'Processed CSI Signal Shape: {signal_processed.shape}')  # Debugging output
        
        # Clear the original signal tensor to free up memory
        del signal_tensor

        # Optional: If memory is tight, you can process in smaller batches here
        if len(processed_signals) >= 2:  # Example: process every 2 signals
            # Convert to tensor and clear the list to free up memory
            torch.stack(processed_signals)  # Convert list to tensor if needed
            processed_signals = []  # Clear to free memory

    # Ensure any remaining signals are processed
    if processed_signals:
        torch.stack(processed_signals)

    # Stack all processed signals into a single tensor
    return torch.stack(processed_signals) if processed_signals else None

# Process the CSI signals
CSI_signals_processed = process_CSI_signals_pytorch(CSI_signals, 6900)

# Check the final shape
print("Processed CSI Signals Shape: ", CSI_signals_processed.shape if CSI_signals_processed is not None else "None")


Original CSI Signal Shape: torch.Size([6600, 245, 2, 2])
Padding CSI signal to length: 6900
Processed CSI Signal Shape: torch.Size([6600, 245, 302, 2])
Original CSI Signal Shape: torch.Size([6000, 245, 2, 2])


/tmp/ipykernel_35856/657104476.py:9: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at ../aten/src/ATen/native/Copy.cpp:305.)
  signal_tensor = torch.tensor(signal, dtype=torch.float32)  # Keep complex parts separate if needed


Padding CSI signal to length: 6900
Processed CSI Signal Shape: torch.Size([6000, 245, 902, 2])


RuntimeError: stack expects each tensor to be equal size, but got [6600, 245, 302, 2] at entry 0 and [6000, 245, 902, 2] at entry 1

In [10]:
import numpy as np

# Load CSI data (Amplitude + Phase)
CSI_data = trainset['CSI']  # Extract the CSI data from your dataset

# Normalize data (GANs usually perform better with normalized inputs)
CSI_data = CSI_data / np.max(np.abs(CSI_data))

# Flatten the data for input into GAN (reshape as needed)
CSI_flattened = CSI_data.reshape(CSI_data.shape[0], -1)  # Reshape each CSI sample into a 1D vector


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (8,) + inhomogeneous part.

In [11]:
# Preprocess Amplitude and Phase
CSI_amplitude = np.abs(CSI_data)
CSI_phase = np.angle(CSI_data)

# Combine features (weighted sum or concatenation)
combined_features = CSI_amplitude + 0.5 * CSI_phase


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (8,) + inhomogeneous part.